In [1]:
import os
import numpy as np

### Exercise 1: Implment 'Multiply' layer.

In [2]:
class AddLayer(object):
    def forward(self,x,y):
        z = x+y
        return z
    def backward(self,dz): #dz represents dL/dz propagated from the above layer
        dx = dz #dL/dz * dz/dx
        dy = dz #dL/dz * dz/dy
        return [dx,dy]

class MultiplyLayer(object):
    def forward(self,x,y):
        z = x*y
        self.x = x  #keep activation during feed forwarding!
        self.y = y
        return z
    def backward(self,dz): #dz represents dL/dz propgated from the above layer
        dx = dz * self.y  #dL/dz * dz/dx
        dy = dz * self.x  #dL/dz * dz/dy
        return [dx,dy]
    def numeric_grad(self,x,y,dz=1.0,eps=0.0001):
        dx = (self.forward(x+eps,y) - self.forward(x-eps,y))/(2*eps) * dz
        dy = (self.forward(x,y+eps) - self.forward(x,y-eps))/(2*eps) * dz
        return [dx,dy]

Feed forward operation.

In [3]:
add = AddLayer()
mult = MultiplyLayer()
add.forward(mult.forward(2,3),-4)

2

Let's see how backprop works.

In [4]:
print(add.backward(1))
print(mult.backward(add.backward(1)[0]))

[1, 1]
[3, 2]


Compare analytical grads with numerical ones.

In [5]:
print(mult.backward(dz=1))
print(mult.numeric_grad(2,3,dz=1,eps=0.01))

[3, 2]
[2.9999999999999805, 1.9999999999999574]


### Exercise 2: implement logistic regression

We use the digit dataset again.

In [6]:
def img2vector(filename):
    returnVect = np.zeros((1,1024)) #images are 32x32, constituting 1024-dim vectors
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadDigits(dataDir):
    labels = []
    fileList = os.listdir(dataDir)
    m = len(fileList)
    dataMat = np.zeros((m,1024))
    labelMat = np.zeros((m,10))
    for i in range(m):
        fileNameStr = fileList[i]  #load the training set
        fileStr = fileNameStr.split('.')[0]  #take off ".txt"
        classNumStr = int(fileStr.split('_')[0])
        labelMat[i,classNumStr]=1.0
        dataMat[i,:] = img2vector('%s/%s' % (dataDir, fileNameStr))
    return dataMat, labelMat

In [7]:
trainMat, trainLabels = loadDigits('trainingDigits') 
testMat, testLabels = loadDigits('testDigits') 
meanVec = np.average(trainMat,axis=0)
stdVec = np.std(trainMat,axis=0)+1.0
trainMat = (trainMat-meanVec)/stdVec
testMat = (testMat-meanVec)/stdVec

TODO: implement Linear layer.

In [8]:
class Linear(object):
    def __init__(self,in_size,out_size):
        self.W = np.random.uniform(size=(in_size,out_size))
        self.b = np.random.uniform(size=(out_size))
        self.X = None
        self.dX = None
        self.db = None
    def forward(self,X):
        self.X = X
        return np.dot(X,self.W) + self.b  
    def backward(self,dY):
        dX = np.dot(dY, self.W.T)
        self.dW = np.dot(self.X.T, dY)
        self.db = np.sum(dY, axis=0)
        return dX


In [9]:
class SoftmaxCrossEnt(object):
    def __init__(self):
        self.Y = None
        self.t = None
    def forward(self,X,t):
        expX = np.exp(X)
        Y = expX / np.sum(expX, axis=1).reshape(-1,1)  #softmax
        L = -np.sum(t*np.log(Y),axis=1)
        self.Y,self.t = Y,t 
        return L
    def backward(self,dZ=None):
        return self.Y - self.t


Now, let's train with an SGD optimizer.

In [10]:
nsample = trainMat.shape[0]  #sample size
bsize=30 #batch size
niter = int(np.ceil(nsample/bsize)) #iteration per epoch
max_epoch=10
alpha = 0.0001 #learning rate

L1 = Linear(1024,10)
Loss = SoftmaxCrossEnt()

for e in range(max_epoch):
    rind = np.random.permutation(nsample) #shuffle data (this is important!)
    trainMat = trainMat[rind]
    trainLabels = trainLabels[rind]
    for i in range(niter):
        X = trainMat[i*bsize:(i+1)*bsize,:]
        t = trainLabels[i*bsize:(i+1)*bsize]        
        L = np.average(Loss.forward(L1.forward(X),t),axis=0)
        
        L1.backward(Loss.backward())
        L1.W = L1.W - alpha*L1.dW
        L1.b = L1.b - alpha*L1.db      
        print("Epoch %d, Iteration %d: Loss %f" % (e+1, i+1, L))

Epoch 1, Iteration 1: Loss 4.159369
Epoch 1, Iteration 2: Loss 4.241496
Epoch 1, Iteration 3: Loss 3.869333
Epoch 1, Iteration 4: Loss 4.371841
Epoch 1, Iteration 5: Loss 4.287650
Epoch 1, Iteration 6: Loss 4.204139
Epoch 1, Iteration 7: Loss 3.842538
Epoch 1, Iteration 8: Loss 4.184414
Epoch 1, Iteration 9: Loss 4.539487
Epoch 1, Iteration 10: Loss 4.133530
Epoch 1, Iteration 11: Loss 4.759192
Epoch 1, Iteration 12: Loss 4.095834
Epoch 1, Iteration 13: Loss 4.782309
Epoch 1, Iteration 14: Loss 4.396527
Epoch 1, Iteration 15: Loss 4.362702
Epoch 1, Iteration 16: Loss 4.099569
Epoch 1, Iteration 17: Loss 4.722175
Epoch 1, Iteration 18: Loss 5.547005
Epoch 1, Iteration 19: Loss 4.653979
Epoch 1, Iteration 20: Loss 3.543349
Epoch 1, Iteration 21: Loss 4.511684
Epoch 1, Iteration 22: Loss 4.297925
Epoch 1, Iteration 23: Loss 3.822228
Epoch 1, Iteration 24: Loss 4.575787
Epoch 1, Iteration 25: Loss 4.613274
Epoch 1, Iteration 26: Loss 4.388602
Epoch 1, Iteration 27: Loss 3.717326
Epoch 1, I

Epoch 4, Iteration 47: Loss 3.625562
Epoch 4, Iteration 48: Loss 1.770071
Epoch 4, Iteration 49: Loss 2.587989
Epoch 4, Iteration 50: Loss 2.456660
Epoch 4, Iteration 51: Loss 3.005008
Epoch 4, Iteration 52: Loss 3.211240
Epoch 4, Iteration 53: Loss 2.492839
Epoch 4, Iteration 54: Loss 2.666094
Epoch 4, Iteration 55: Loss 2.912614
Epoch 4, Iteration 56: Loss 2.961146
Epoch 4, Iteration 57: Loss 2.788989
Epoch 4, Iteration 58: Loss 3.471017
Epoch 4, Iteration 59: Loss 3.288556
Epoch 4, Iteration 60: Loss 3.010062
Epoch 4, Iteration 61: Loss 2.541403
Epoch 4, Iteration 62: Loss 2.745540
Epoch 4, Iteration 63: Loss 3.247861
Epoch 4, Iteration 64: Loss 3.067235
Epoch 4, Iteration 65: Loss 3.917911
Epoch 5, Iteration 1: Loss 3.454386
Epoch 5, Iteration 2: Loss 3.164608
Epoch 5, Iteration 3: Loss 2.402362
Epoch 5, Iteration 4: Loss 2.732642
Epoch 5, Iteration 5: Loss 3.178726
Epoch 5, Iteration 6: Loss 2.364411
Epoch 5, Iteration 7: Loss 3.846074
Epoch 5, Iteration 8: Loss 1.973493
Epoch 5, 

Epoch 10, Iteration 51: Loss 1.796119
Epoch 10, Iteration 52: Loss 1.462009
Epoch 10, Iteration 53: Loss 1.715597
Epoch 10, Iteration 54: Loss 1.232431
Epoch 10, Iteration 55: Loss 1.543254
Epoch 10, Iteration 56: Loss 1.232938
Epoch 10, Iteration 57: Loss 2.240966
Epoch 10, Iteration 58: Loss 1.255948
Epoch 10, Iteration 59: Loss 1.650053
Epoch 10, Iteration 60: Loss 1.786854
Epoch 10, Iteration 61: Loss 2.135444
Epoch 10, Iteration 62: Loss 1.308016
Epoch 10, Iteration 63: Loss 1.654336
Epoch 10, Iteration 64: Loss 1.863353
Epoch 10, Iteration 65: Loss 1.577022


### Exercise 3: implement 3-layer perceptron

TODO: implement sigmoid activation function

In [11]:
class Sigmoid(object):
    def __init__(self):
        self.Y = None
    def forward(self,X):
        self.Y = 1 / (1 + np.exp(-X))
        return self.Y  
    def backward(self,dY):
        dX = dY * (1.0 - self.Y)*self.Y
        return dX


TODO: implement the network and solver

In [12]:
nsample = trainMat.shape[0]  #sample size
bsize=30 #batch size
niter = int(np.ceil(nsample/bsize)) #iteration per epoch
max_epoch=10
alpha = 0.001 #learning rate

L1 = Linear(1024,200)
Sig1 = Sigmoid()
L2 = Linear(200,10)
Loss = SoftmaxCrossEnt()
Layers = [L1,Sig1,L2,Loss]

for e in range(max_epoch):
    rind = np.random.permutation(nsample) #shuffle data (this is important!)
    trainMat = trainMat[rind]
    trainLabels = trainLabels[rind]
    for i in range(niter):
        X = trainMat[i*bsize:(i+1)*bsize,:]
        t = trainLabels[i*bsize:(i+1)*bsize]
        h = X
        for l,layer in enumerate(Layers):
            if l<len(Layers)-1:
                h = layer.forward(h)
            else:
                h = layer.forward(h,t) #the last layer: loss
        L = np.average(h,axis=0)

        dZ = None
        for l,layer in enumerate(reversed(Layers)):
            dZ = layer.backward(dZ)
        L1.W = L1.W - alpha*L1.dW
        L1.b = L1.b - alpha*L1.db      
        L2.W = L2.W - alpha*L2.dW
        L2.b = L2.b - alpha*L2.db

        print("Epoch %d, Iteration %d: Loss %f" % (e+1, i+1, L))

Epoch 1, Iteration 1: Loss 2.790536
Epoch 1, Iteration 2: Loss 2.782389
Epoch 1, Iteration 3: Loss 3.645079
Epoch 1, Iteration 4: Loss 3.015634
Epoch 1, Iteration 5: Loss 2.603374
Epoch 1, Iteration 6: Loss 2.934077
Epoch 1, Iteration 7: Loss 2.878302
Epoch 1, Iteration 8: Loss 2.716823
Epoch 1, Iteration 9: Loss 2.808855
Epoch 1, Iteration 10: Loss 2.523588
Epoch 1, Iteration 11: Loss 2.299271
Epoch 1, Iteration 12: Loss 2.552672
Epoch 1, Iteration 13: Loss 2.729319
Epoch 1, Iteration 14: Loss 2.387571
Epoch 1, Iteration 15: Loss 2.416777
Epoch 1, Iteration 16: Loss 2.417146
Epoch 1, Iteration 17: Loss 2.445489
Epoch 1, Iteration 18: Loss 2.246107
Epoch 1, Iteration 19: Loss 2.453499
Epoch 1, Iteration 20: Loss 2.489318
Epoch 1, Iteration 21: Loss 2.328210
Epoch 1, Iteration 22: Loss 2.433568
Epoch 1, Iteration 23: Loss 2.631105
Epoch 1, Iteration 24: Loss 2.462755
Epoch 1, Iteration 25: Loss 2.546251
Epoch 1, Iteration 26: Loss 2.385079
Epoch 1, Iteration 27: Loss 2.176079
Epoch 1, I

Epoch 4, Iteration 50: Loss 2.012962
Epoch 4, Iteration 51: Loss 2.166887
Epoch 4, Iteration 52: Loss 2.149477
Epoch 4, Iteration 53: Loss 2.032506
Epoch 4, Iteration 54: Loss 2.036394
Epoch 4, Iteration 55: Loss 2.220647
Epoch 4, Iteration 56: Loss 2.099929
Epoch 4, Iteration 57: Loss 1.937098
Epoch 4, Iteration 58: Loss 2.021318
Epoch 4, Iteration 59: Loss 2.235699
Epoch 4, Iteration 60: Loss 1.918243
Epoch 4, Iteration 61: Loss 2.071710
Epoch 4, Iteration 62: Loss 2.179685
Epoch 4, Iteration 63: Loss 2.062874
Epoch 4, Iteration 64: Loss 2.130973
Epoch 4, Iteration 65: Loss 2.228512
Epoch 5, Iteration 1: Loss 2.129535
Epoch 5, Iteration 2: Loss 2.089141
Epoch 5, Iteration 3: Loss 2.343943
Epoch 5, Iteration 4: Loss 2.161075
Epoch 5, Iteration 5: Loss 2.214439
Epoch 5, Iteration 6: Loss 1.985574
Epoch 5, Iteration 7: Loss 2.036767
Epoch 5, Iteration 8: Loss 2.026332
Epoch 5, Iteration 9: Loss 2.173990
Epoch 5, Iteration 10: Loss 2.197929
Epoch 5, Iteration 11: Loss 2.213362
Epoch 5, I

Epoch 8, Iteration 47: Loss 1.834238
Epoch 8, Iteration 48: Loss 2.072485
Epoch 8, Iteration 49: Loss 2.049918
Epoch 8, Iteration 50: Loss 1.943270
Epoch 8, Iteration 51: Loss 1.975621
Epoch 8, Iteration 52: Loss 1.821301
Epoch 8, Iteration 53: Loss 2.112495
Epoch 8, Iteration 54: Loss 1.856301
Epoch 8, Iteration 55: Loss 2.177879
Epoch 8, Iteration 56: Loss 1.696245
Epoch 8, Iteration 57: Loss 1.701542
Epoch 8, Iteration 58: Loss 1.884084
Epoch 8, Iteration 59: Loss 1.917514
Epoch 8, Iteration 60: Loss 1.984295
Epoch 8, Iteration 61: Loss 1.892281
Epoch 8, Iteration 62: Loss 2.021053
Epoch 8, Iteration 63: Loss 1.977479
Epoch 8, Iteration 64: Loss 1.987411
Epoch 8, Iteration 65: Loss 2.103672
Epoch 9, Iteration 1: Loss 2.244736
Epoch 9, Iteration 2: Loss 1.857798
Epoch 9, Iteration 3: Loss 2.164953
Epoch 9, Iteration 4: Loss 1.967336
Epoch 9, Iteration 5: Loss 1.873585
Epoch 9, Iteration 6: Loss 2.062077
Epoch 9, Iteration 7: Loss 1.872245
Epoch 9, Iteration 8: Loss 1.968691
Epoch 9, 